# AI-Based Incident Categorization and Resolution Recommendation System

## Problem Definition
Retail IT support teams receive thousands of incident tickets daily.
Manual categorization and resolution selection increases response time and errors.

## Objective
- Predict incident category using NLP and Machine Learning
- Recommend possible resolution based on similar past incidents

## Real-World Relevance
Faster ticket routing improves store uptime and reduces business losses in retail operations.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df = pd.read_excel(r"C:\Users\manu.chopra\OneDrive - Reliance Corporate IT Park Limited\Documents\ITSM_AI_Project\tickets.xlsx")
df.head()


,INCIDENT_ID,STATUS,AGE,OPEN_TIME_IM,OPEN_TIME_SD,TITLE,DESCRIPTION,ASSIGNMENT,ASSIGNEE,CONTACT_NAME,...,CATEGORY,SUBCATEGORY,ISSUE_TYPE,UPDATE_TIME,RESOLVE_TIME,CLOSE_TIME,RESOLUTION_CODE,RESOLUTION,TICKET_REOPEN_COUNT,VENDOR
0,IT17820042,CLOSED,69,2025-05-03 08:32:57,2025-05-03 08:09:05,Charging not happend,Dear Team Kindly help to resolve there is ch...,SE_UP,Limuty,Jeky,...,LAPTOP,SYSTEM ISSUE,POWER ADAPTER,2025-07-14 11:31:30,2025-07-11 18:37:49,2025-07-14 11:31:30,VENDOR SUPPORT,Rc Laptop Issue - Spare Laptop And Power Ada...,0,LENOVO
1,IT17878519,CLOSED,37,2025-05-30 19:05:47,2025-05-30 16:31:04,LAPTOP NOT TURNING ON,Dear Team laptop is not working Adaptor is...,SL_CHE,Dily,Fybo,...,LAPTOP,SYSTEM ISSUE,POWER ADAPTER,2025-07-10 11:31:44,2025-07-07 10:30:51,2025-07-10 11:31:44,SPARE CALL,Rc Laptop Not Working Issue - Spare La...,0,HP
2,IT17888923,CLOSED,29,2025-06-05 14:22:37,2025-06-05 12:49:39,Laptop Faulty,laptop Led Screen replacement required as top ...,SE_OR,Podiky,Byi,...,LAPTOP,SYSTEM ISSUE,BATTERY ISSUE,2025-07-07 11:31:35,2025-07-04 21:01:26,2025-07-07 11:31:35,VENDOR SUPPORT,Rc Battery Backup Issue - As Discussed With U...,0,LENOVO
3,IT17889337,CLOSED,28,2025-06-05 16:00:33,2025-06-04 19:10:44,LAPTOP PROBLEM,LAPTOP NOT SWITCHING ON,SE_KA,Vyciwa,Zygywu,...,LAPTOP,SYSTEM ISSUE,BATTERY ISSUE,2025-07-07 11:31:07,2025-07-04 16:00:00,2025-07-07 11:31:07,VENDOR SUPPORT,Rc Laptop Having Battery Issue - Remotely ...,0,HP
4,IT17897240,CLOSED,23,2025-06-09 15:07:54,2025-06-09 14:33:49,Paper Jam,Printer model Epson M205 System IP NP Sinc...,SL_IN,Kudo,Cemi,...,PRINTERS,DESKJET PRINTER,HARDWARE FAULTY,2025-07-06 11:31:07,2025-07-03 14:02:24,2025-07-06 11:31:07,UNIT REPLACEMENT CALL,Rc Epson Printer Is Not Working - Servi...,0,EPSON


In [3]:
print(df.shape)
print(df.columns)
df.isnull().sum()


(1849, 28)
Index(['INCIDENT_ID', 'STATUS', 'AGE', 'OPEN_TIME_IM', 'OPEN_TIME_SD', 'TITLE',
       'DESCRIPTION', 'ASSIGNMENT', 'ASSIGNEE', 'CONTACT_NAME', 'PHONE_NUMBER',
       'LOCATION', 'STORENAME', 'STOREFORMAT', 'MAIN_FORMAT', 'STORECITY',
       'STORESTATE', 'FORMAT', 'CATEGORY', 'SUBCATEGORY', 'ISSUE_TYPE',
       'UPDATE_TIME', 'RESOLVE_TIME', 'CLOSE_TIME', 'RESOLUTION_CODE',
       'RESOLUTION', 'TICKET_REOPEN_COUNT', 'VENDOR'],
      dtype='object')


INCIDENT_ID              0
STATUS                   0
AGE                      0
OPEN_TIME_IM             0
OPEN_TIME_SD             0
TITLE                    0
DESCRIPTION              0
ASSIGNMENT               0
ASSIGNEE                 0
CONTACT_NAME             0
PHONE_NUMBER             0
LOCATION                 0
STORENAME                0
STOREFORMAT              0
MAIN_FORMAT              0
STORECITY                0
STORESTATE               0
FORMAT                   0
CATEGORY                 0
SUBCATEGORY              0
ISSUE_TYPE               0
UPDATE_TIME              0
RESOLVE_TIME             0
CLOSE_TIME             140
RESOLUTION_CODE          0
RESOLUTION               0
TICKET_REOPEN_COUNT      0
VENDOR                 617
dtype: int64

In [4]:
df["TEXT"] = df["TITLE"].fillna("") + " " + df["DESCRIPTION"].fillna("")


In [5]:
df = df.dropna(subset=["CATEGORY"])
print(df.shape)


(1849, 29)


In [6]:
X = df["TEXT"]
y = df["CATEGORY"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [7]:
vectorizer = TfidfVectorizer(
    max_features=15000,
    stop_words="english",
    ngram_range=(1,2)
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [8]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)


LogisticRegression(max_iter=1000)

In [9]:
pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))


Accuracy: 0.9675675675675676
              precision    recall  f1-score   support

      LAPTOP       0.99      0.98      0.99       143
     PRINTER       0.00      0.00      0.00         8
    PRINTERS       0.95      1.00      0.97       219

    accuracy                           0.97       370
   macro avg       0.65      0.66      0.65       370
weighted avg       0.95      0.97      0.96       370



C:\Users\manu.chopra\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\manu.chopra\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\manu.chopra\AppData\Roaming\Python\Python313\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metr

In [11]:
all_vec = vectorizer.transform(df["TEXT"])

def recommend_resolution(issue_text):
    vec = vectorizer.transform([issue_text])
    sim = cosine_similarity(vec, all_vec)
    idx = sim.argmax()
    return df.iloc[idx]["RESOLUTION"]

test_issue = "printer not working"
print("Suggested Resolution:")
print(recommend_resolution(test_issue))


Suggested Resolution:
Km2985 Rc   At Printer Not Working -  Check And Found Printer Blank Print So Vendor  Epson Egg Visited The Site And Resolved  The Issue Same Has Confirm With User 


## Ethical Considerations and Responsible AI

- Model depends on historical data quality.
- Biased routing may occur if training data is imbalanced.
- Recommendations should assist humans, not fully automate decisions.
- Sensitive personal data must be protected.


## Conclusion and Future Scope

The project demonstrates practical use of NLP in IT Service Management.
Future improvements can include:
- Deep learning embeddings
- LLM-based explanation of resolutions
- Auto-routing and automation integration
